<a href="https://colab.research.google.com/github/a-mhamdi/nlp-unraveled/blob/main/05_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Analysis and Advanced Techniques
---

**Outlines**
----
1. [Text classification](#text-classification)
1. [Named entity recognition](#ner)
1. [Sentiment analysis](#sentiment-analysis)
1. [Word embeddings](#word-embeddings)

## Text classification <a name="text-classification"></a>


## Named entity recognition (NER) <a name="ner"></a>


## Sentiment analysis <a name="sentiment-analysis"></a>


## Word embeddings <a name="word-embeddings"></a>